# 🚀 Full HDFS Dataset Training

**Data:** LogHub_HDFS/HDFS_v1/preprocessed/Event_traces.csv (125MB)

**Config:** Same as before, just more data

**Output:** `output/`

In [ ]:
!pip install seaborn -q
!mkdir -p output

In [ ]:
import os, math, random, time, json, gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️ Device: {device}")
if device == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# Configuration
SEEDS = [42, 123, 456]  # 3 seeds for speed
CONTEXT_LEN = 128
EPOCHS = 10
BATCH_SIZE = 256  # Larger batch for more data
LAMBDA_VHM = 0.2
LR = 1e-4
MASK_RATIO = 0.15
USE_AMP = True
OUTPUT_DIR = 'output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Load FULL HDFS dataset
# Try multiple paths
PATHS = [
    'content/Event_traces.csv',  # Lightning AI
    '/content/Event_traces.csv',  # Colab
    'Event_traces.csv'
]

HDFS_PATH = None
for p in PATHS:
    if os.path.exists(p):
        HDFS_PATH = p
        break

if not HDFS_PATH:
    raise FileNotFoundError("Upload Event_traces.csv (125MB) to content/")

print(f"📁 Loading: {HDFS_PATH}")
df = pd.read_csv(HDFS_PATH)
print(f"📊 Total rows: {len(df):,}")

In [ ]:
# Parse sessions
raw, labels = [], []
for _, r in df.iterrows():
    f = str(r['Features']).strip('[]')
    s = [e.strip().strip("'") for e in f.split(',') if e.strip()]
    if len(s) >= 2:
        raw.append(s)
        labels.append(0 if 'success' in str(r['Label']).lower() else 1)

labels = np.array(labels)
print(f"📊 Valid sessions: {len(raw):,}")
print(f"   Anomalies: {labels.sum():,} ({labels.mean()*100:.1f}%)")

In [ ]:
# Build vocab
e2i = {'[PAD]': 0, '[UNK]': 1, '[CLS]': 2, '[MASK]': 3}
for i, e in enumerate(sorted(set(e for s in raw for e in s))):
    e2i[e] = i + 4
V = len(e2i)
print(f"📝 Vocab size: {V}")

def encode(seq):
    enc = [e2i.get(e, 1) for e in seq][:CONTEXT_LEN]
    return enc + [0] * (CONTEXT_LEN - len(enc))

print("Encoding...")
X = np.array([encode(s) for s in raw])
print(f"   Shape: {X.shape}")

In [ ]:
# Split
np.random.seed(0)
idx = np.random.permutation(len(X))
train_end = int(len(X) * 0.7)
test_idx = idx[int(len(X) * 0.85):]
train_idx = idx[:train_end]
train_idx = train_idx[labels[train_idx] == 0]  # Normal only

X_train, X_test = X[train_idx], X[test_idx]
y_test = labels[test_idx]

print(f"📊 Split:")
print(f"   Train (normal): {len(X_train):,}")
print(f"   Test: {len(X_test):,} (anomaly: {y_test.sum():,})")

In [ ]:
class MaskedLMDataset(Dataset):
    def __init__(self, X, V, mask_ratio=0.15):
        self.X, self.V, self.mask_ratio = X, V, mask_ratio
    def __len__(self): return len(self.X)
    def __getitem__(self, i):
        orig = np.clip(self.X[i].copy(), 0, self.V-1)
        masked, labels = orig.copy(), np.full_like(orig, -100)
        nz = np.where(orig != 0)[0]
        n_mask = max(1, int(len(nz) * self.mask_ratio))
        if len(nz) > 0:
            for p in np.random.choice(nz, min(n_mask, len(nz)), replace=False):
                labels[p], masked[p] = orig[p], 3
        return torch.LongTensor(masked), torch.LongTensor(orig), torch.LongTensor(labels)

class PositionalEncoding(nn.Module):
    def __init__(self, d, max_len=512):
        super().__init__()
        pe = torch.zeros(max_len, d)
        pos = torch.arange(max_len).unsqueeze(1).float()
        div = torch.exp(torch.arange(0, d, 2).float() * (-math.log(10000) / d))
        pe[:, 0::2], pe[:, 1::2] = torch.sin(pos * div), torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self, x): return x + self.pe[:, :x.size(1)]

class LogBERTVHM(nn.Module):
    def __init__(self, V, d=128, h=4, L=4):
        super().__init__()
        self.d, self.V = d, V
        self.emb = nn.Embedding(V, d, padding_idx=0)
        self.pos = PositionalEncoding(d)
        self.tf = nn.TransformerEncoder(nn.TransformerEncoderLayer(d, h, d*4, 0.1, batch_first=True), L)
        self.fc = nn.Linear(d, V)
        self.register_buffer('center', torch.zeros(d))
    def forward(self, x):
        x = torch.clamp(x, 0, self.V-1)
        return self.fc(self.tf(self.pos(self.emb(x) * math.sqrt(self.d)), src_key_padding_mask=(x==0)))
    def get_emb(self, x):
        x = torch.clamp(x, 0, self.V-1)
        m = (~(x==0)).unsqueeze(-1).float()
        return (self.tf(self.pos(self.emb(x) * math.sqrt(self.d)), src_key_padding_mask=(x==0)) * m).sum(1) / m.sum(1).clamp(min=1)

In [ ]:
def train_model(seed):
    t0 = time.time()
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    
    dl = DataLoader(MaskedLMDataset(X_train, V, MASK_RATIO), BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    model = LogBERTVHM(V).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=LR)
    crit = nn.CrossEntropyLoss(ignore_index=-100)
    scaler = GradScaler() if USE_AMP else None
    
    # Init center (sample for speed)
    model.eval()
    with torch.no_grad():
        sample_dl = DataLoader(MaskedLMDataset(X_train[:5000], V), 256)
        model.center = torch.cat([model.get_emb(o.to(device)) for _, o, _ in sample_dl]).mean(0)
    
    model.train()
    for ep in range(EPOCHS):
        loss_sum, n_batch = 0, 0
        for m, o, l in dl:
            m, o, l = m.to(device), o.to(device), l.to(device)
            opt.zero_grad()
            if USE_AMP:
                with autocast():
                    lo = model(m)
                    loss = crit(lo.view(-1, V), l.view(-1)) + LAMBDA_VHM * ((model.get_emb(o) - model.center)**2).sum(1).mean()
                scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
            else:
                lo = model(m)
                loss = crit(lo.view(-1, V), l.view(-1)) + LAMBDA_VHM * ((model.get_emb(o) - model.center)**2).sum(1).mean()
                loss.backward(); opt.step()
            with torch.no_grad(): model.center = 0.99 * model.center + 0.01 * model.get_emb(o).mean(0)
            loss_sum += loss.item(); n_batch += 1
        print(f"  Epoch {ep+1}/{EPOCHS}: Loss={loss_sum/n_batch:.4f}")
    return model, (time.time() - t0) / 60

def score_sequences(model, X_data, n_samples=5):
    model.eval()
    scores = []
    for seq in X_data:
        seq = np.clip(seq, 0, V-1)
        nz = seq[seq != 0]
        if len(nz) < 2: scores.append(0); continue
        pos = np.random.choice(len(nz), min(n_samples, len(nz)), replace=False)
        mlm = 0
        with torch.no_grad():
            for p in pos:
                m = nz.copy(); tok = m[p]; m[p] = 3
                inp = np.zeros(CONTEXT_LEN, dtype=np.int64); inp[:len(m)] = m
                lo = model(torch.LongTensor(inp).unsqueeze(0).to(device))
                mlm += -math.log(F.softmax(lo[0, p].float(), -1)[tok].item() + 1e-10)
            mlm /= len(pos)
            inp = np.zeros(CONTEXT_LEN, dtype=np.int64); inp[:len(nz)] = nz
            emb = model.get_emb(torch.LongTensor(inp).unsqueeze(0).to(device))
            vhm = ((emb - model.center)**2).sum().item()
        scores.append(mlm + 0.1 * vhm)
    return np.array(scores)

def eval_f1(sc, y):
    best = {'F1': 0, 'P': 0, 'R': 0, 'th': 0}
    for pct in range(30, 100):
        th = np.percentile(sc, pct)
        pr = (sc > th).astype(int)
        tp, fp, fn = ((pr==1)&(y==1)).sum(), ((pr==1)&(y==0)).sum(), ((pr==0)&(y==1)).sum()
        P, R = tp/(tp+fp+1e-10), tp/(tp+fn+1e-10)
        F1 = 2*P*R/(P+R+1e-10)
        if F1 > best['F1']: best = {'F1': F1, 'P': P, 'R': R, 'th': th}
    return best

In [ ]:
# Train
all_scores, all_results = [], []

for i, seed in enumerate(SEEDS):
    print(f"\n{'='*50}\n🌱 Seed {seed} ({i+1}/{len(SEEDS)})\n{'='*50}")
    model, t = train_model(seed)
    
    print("  Scoring...")
    sc = score_sequences(model, X_test)
    all_scores.append(sc)
    
    r = eval_f1(sc, y_test)
    print(f"  ✅ F1={r['F1']:.1%}, P={r['P']:.1%}, R={r['R']:.1%}, Time={t:.1f}min")
    all_results.append({'seed': seed, 'time': t, **r})
    
    np.save(f'{OUTPUT_DIR}/scores_seed_{seed}.npy', sc)
    pd.DataFrame(all_results).to_csv(f'{OUTPUT_DIR}/results.csv', index=False)
    del model; torch.cuda.empty_cache(); gc.collect()

print("\n✅ Training Complete")

In [ ]:
# Ensemble
print("\n" + "="*50 + "\n🎯 ENSEMBLE RESULTS\n" + "="*50)

ens = np.mean(all_scores, axis=0)
np.save(f'{OUTPUT_DIR}/scores_ensemble.npy', ens)

r = eval_f1(ens, y_test)
print(f"\n📊 Individual Avg F1: {np.mean([x['F1'] for x in all_results]):.1%}")
print(f"📊 Ensemble F1: {r['F1']:.1%}, P={r['P']:.1%}, R={r['R']:.1%}")

f1s = [x['F1'] for x in all_results]
mean_f1, std_f1 = np.mean(f1s), np.std(f1s)
print(f"\n📈 95% CI: {mean_f1:.1%} ± {1.96*std_f1/np.sqrt(len(f1s)):.1%}")

In [ ]:
# Save
final = {
    'dataset': 'HDFS_full',
    'data_size': {'train': len(X_train), 'test': len(X_test)},
    'config': {'seeds': SEEDS, 'epochs': EPOCHS, 'batch': BATCH_SIZE},
    'individual': all_results,
    'ensemble_f1': eval_f1(ens, y_test),
    'comparison': {
        'small_dataset_ensemble': 0.763,
        'full_dataset_ensemble': r['F1']
    }
}
with open(f'{OUTPUT_DIR}/final_results.json', 'w') as f:
    json.dump(final, f, indent=2, default=float)

print(f"\n✅ Saved to {OUTPUT_DIR}/")
print(f"\n📊 COMPARISON:")
print(f"   Small dataset (100k): 76.3% F1")
print(f"   Full dataset ({len(X_train)+len(X_test):,}): {r['F1']:.1%} F1")